# Clustering mit DBSCAN

Zusammenhängende Gebiete in OpenStreetMap erkennen

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import cluster, preprocessing
import requests

Daten für Overpass-API-Abfrage

- Beschreibung: [Wiki](https://wiki.openstreetmap.org/wiki/Overpass_API)
- Server: [overpass-api.de](https://overpass-api.de/api/interpreter)
- Interaktiver Test: [overpass-turbo](https://overpass-turbo.eu)

In [ ]:
query = '''
[bbox: 50.828, 14.043, 50.838, 14.059];
way[natural=cliff];
>;
out;
'''
# bbox: lat0, lon0, lat1, lon1
# dlat: 0.01, dlon: 0.016
URL = 'https://overpass-api.de/api/interpreter'

POST-Request

In [ ]:
r = requests.post(url=URL, data={'data': query})
#print(r.text)

Umwandeln in Data Frame. Nur für einfach strukturierte Daten geeignet!

In [ ]:
df = pd.read_xml(r.content, xpath='./node', parser='etree')
df

Graphische Kontrolle

In [ ]:
df.plot('lon', 'lat', kind='scatter')

Skalierung (naiv).

Korrekte Skalierung über Berechnung der [Orthodrome](https://de.wikipedia.org/wiki/Orthodrome):
- Bestimme Abstand zwischen linken und rechten bzw. zwischen oberen und unteren Rand der Bounding Box.
- Berechne die Skalierungsfaktoren für x- und y-Richtung und skaliere mit diesen die Daten.

In [ ]:
X = df.loc[:, ['lon', 'lat']].values
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

Kontrollanzeige skalierte Daten

In [ ]:
plt.scatter(X[:, 0], X[:, 1], alpha=0.5)

KMeans Clustering

In [ ]:
K = 8
kmeans = cluster.KMeans(n_clusters=K)
labels = kmeans.fit_predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='rainbow', alpha=0.5)

DBScan

In [ ]:
dbscan = cluster.DBSCAN(eps=0.5)
labels = dbscan.fit_predict(X)
# Ermittelte Label und Anzahl zugehöriger Elemente)
u, c = np.unique(labels, return_counts=True)
u, c

Anzeige der Cluster

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='rainbow', alpha=0.5)

Suche optimales Epsilon

In [ ]:
eps_list = np.arange(0.1, 1.0, 0.1)
nr = 0
plt.figure(figsize=(15, 15))
for eps in eps_list:
    nr += 1
    plt.subplot(3, 3, nr)
    dbscan = cluster.DBSCAN(eps=eps)
    labels = dbscan.fit_predict(X)
    cl_count = len(np.unique(labels))
    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='rainbow', alpha=0.5)
    plt.title('eps: {:.1} cluster: {}'.format(eps, cl_count))